In [1]:
# Celda 1: Definición de Librerías
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import logging

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("001_load_airbnb.log"),
        logging.StreamHandler()
    ]
)

logging.info("Inicio del notebook de la carga 001_load_airbnb")

2025-05-17 14:34:37,206 - INFO - Inicio del notebook de la carga 001_load_airbnb


In [4]:
# Celda 3: Definición de Variables Constantes y Carga de .env
CSV_FILE_PATH = '/home/y4xul/Proyecto_ETL/data/raw/Airbnb_Open_Data.csv'
ENV_FILE_PATH = '/home/y4xul/Proyecto_ETL/env/.env'
TABLE_NAME = 'raw_airbnb' # Nombre de la tabla en PostgreSQL

logging.info(f"Ruta del archivo CSV: {CSV_FILE_PATH}")
logging.info(f"Ruta del archivo .env: {ENV_FILE_PATH}")
logging.info(f"Nombre de la tabla en PostgreSQL: {TABLE_NAME}")

# Cargar variables de entorno
load_dotenv(ENV_FILE_PATH)

POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_DATABASE = os.getenv('POSTGRES_DATABASE')

if not all([POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_HOST, POSTGRES_PORT, POSTGRES_DATABASE]):
    logging.error("Una o más variables de entorno de PostgreSQL no están definidas. Revisa tu archivo .env.")
    # Puedes decidir si detener el script aquí o continuar con valores por defecto (no recomendado para producción)
    raise ValueError("Faltan credenciales de PostgreSQL en el archivo .env")
else:
    logging.info("Variables de entorno para PostgreSQL cargadas correctamente.")

2025-05-17 14:37:01,653 - INFO - Ruta del archivo CSV: /home/y4xul/Proyecto_ETL/data/raw/Airbnb_Open_Data.csv
2025-05-17 14:37:01,657 - INFO - Ruta del archivo .env: /home/y4xul/Proyecto_ETL/env/.env
2025-05-17 14:37:01,660 - INFO - Nombre de la tabla en PostgreSQL: raw_airbnb
2025-05-17 14:37:01,669 - ERROR - Una o más variables de entorno de PostgreSQL no están definidas. Revisa tu archivo .env.


ValueError: Faltan credenciales de PostgreSQL en el archivo .env

In [4]:
# Celda 4: Pre-definición del DataFrame
df_airbnb = pd.DataFrame()
logging.info("DataFrame 'df_airbnb' predefinido como un DataFrame vacío.")

2025-05-16 22:04:18,396 - INFO - DataFrame 'df_airbnb' predefinido como un DataFrame vacío.


In [5]:
# Celda 5: Carga del DataFrame desde el archivo CSV
try:
    logging.info(f"Intentando cargar el archivo CSV: {CSV_FILE_PATH}")
    df_airbnb = pd.read_csv(CSV_FILE_PATH, low_memory=False)
    logging.info(f"Archivo CSV '{CSV_FILE_PATH}' cargado exitosamente.")
    logging.info(f"El DataFrame tiene {df_airbnb.shape[0]} filas y {df_airbnb.shape[1]} columnas.")
except FileNotFoundError:
    logging.error(f"Error: Archivo CSV no encontrado en '{CSV_FILE_PATH}'")
    raise
except Exception as e:
    logging.error(f"Ocurrió un error al cargar el CSV '{CSV_FILE_PATH}': {e}")
    raise

2025-05-16 22:04:18,402 - INFO - Intentando cargar el archivo CSV: /home/nicolas/Escritorio/proyecto ETL/develop/data/raw/Airbnb_Open_Data.csv
2025-05-16 22:04:18,835 - INFO - Archivo CSV '/home/nicolas/Escritorio/proyecto ETL/develop/data/raw/Airbnb_Open_Data.csv' cargado exitosamente.
2025-05-16 22:04:18,835 - INFO - El DataFrame tiene 102599 filas y 26 columnas.


In [6]:
# Celda 6: Verificación de la carga del DataFrame
if not df_airbnb.empty:
    logging.info("Mostrando las primeras 5 filas del DataFrame df_airbnb (formato markdown):")
    logging.info(f"\n{df_airbnb.head().to_markdown(index=False)}")
else:
    logging.warning("El DataFrame df_airbnb está vacío. No se puede mostrar el head.")

2025-05-16 22:04:18,840 - INFO - Mostrando las primeras 5 filas del DataFrame df_airbnb (formato markdown):
2025-05-16 22:04:18,847 - INFO - 
|      id | NAME                                             |     host id | host_identity_verified   | host name   | neighbourhood group   | neighbourhood   |     lat |     long | country       | country code   | instant_bookable   | cancellation_policy   | room type       |   Construction year | price   | service fee   |   minimum nights |   number of reviews | last review   |   reviews per month |   review rate number |   calculated host listings count |   availability 365 | house_rules                                                                                                                                                                                                                                                                                                                                                                            

In [7]:
# Celda 7: Crear motor de conexión a PostgreSQL
if not df_airbnb.empty:
    try:
        DATABASE_URL = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DATABASE}"
        engine = create_engine(DATABASE_URL)
        logging.info("Motor de SQLAlchemy para PostgreSQL creado.")
        
        with engine.connect() as connection:
            logging.info("Conexión a la base de datos PostgreSQL establecida y verificada exitosamente.")
    except Exception as e:
        logging.error(f"Error al crear el motor de SQLAlchemy o al conectar a PostgreSQL: {e}")
        engine = None
else:
    logging.warning("El DataFrame está vacío, se omite la creación del motor de DB.")

2025-05-16 22:04:18,871 - INFO - Motor de SQLAlchemy para PostgreSQL creado.
DETAIL:  La base de datos fue creada usando la versión de ordenamiento 2.31, pero el sistema operativo provee la versión 2.35.
HINT:  Reconstruya todos los objetos en esta base de datos que usen el ordenamiento por omisión y ejecute ALTER DATABASE airbnb REFRESH COLLATION VERSION, o construya PostgreSQL con la versión correcta de la biblioteca.
2025-05-16 22:04:18,881 - INFO - Conexión a la base de datos PostgreSQL establecida y verificada exitosamente.


In [8]:
# Celda 8: Crear tabla vacía en PostgreSQL (si no existe o reemplazarla)
if not df_airbnb.empty and 'engine' in locals() and engine is not None:
    try:
        logging.info(f"Intentando crear/reemplazar la tabla vacía '{TABLE_NAME}' en PostgreSQL.")
        df_airbnb.head(0).to_sql(TABLE_NAME, engine, if_exists='replace', index=False)
        logging.info(f"Tabla '{TABLE_NAME}' creada/reemplazada exitosamente (estructura definida, sin datos).")
    except Exception as e:
        logging.error(f"Error al crear la tabla '{TABLE_NAME}' en PostgreSQL: {e}")
else:
    if df_airbnb.empty:
        logging.warning("DataFrame está vacío. No se intentará crear la tabla en la base de datos.")
    if 'engine' not in locals() or engine is None:
        logging.warning("Motor de base de datos no inicializado. No se intentará crear la tabla.")

2025-05-16 22:04:18,889 - INFO - Intentando crear/reemplazar la tabla vacía 'raw_airbnb' en PostgreSQL.
2025-05-16 22:04:18,918 - INFO - Tabla 'raw_airbnb' creada/reemplazada exitosamente (estructura definida, sin datos).


In [9]:
# Celda 9: Insertar el DataFrame en la tabla de PostgreSQL
if not df_airbnb.empty and 'engine' in locals() and engine is not None:
    try:
        logging.info(f"Intentando insertar datos del DataFrame en la tabla '{TABLE_NAME}'.")
        df_airbnb.to_sql(TABLE_NAME, engine, if_exists='append', index=False)
        logging.info(f"Datos insertados exitosamente en la tabla '{TABLE_NAME}'.")
    except Exception as e:
        logging.error(f"Error al insertar datos en la tabla '{TABLE_NAME}': {e}")
else:
    if df_airbnb.empty:
        logging.warning("DataFrame está vacío. No hay datos para insertar en la base de datos.")
    if 'engine' not in locals() or engine is None:
        logging.warning("Motor de base de datos no inicializado. No se pueden insertar datos.")

2025-05-16 22:04:18,924 - INFO - Intentando insertar datos del DataFrame en la tabla 'raw_airbnb'.
2025-05-16 22:04:24,674 - INFO - Datos insertados exitosamente en la tabla 'raw_airbnb'.


In [10]:
# Celda 10: Verificar los primeros 5 datos desde la base de datos PostgreSQL
if 'engine' in locals() and engine is not None:
    try:
        logging.info(f"Consultando las primeras 5 filas de la tabla '{TABLE_NAME}' desde PostgreSQL.")
        query = f"SELECT * FROM \"{TABLE_NAME}\" LIMIT 5;"
        df_from_db = pd.read_sql_query(query, engine)
        logging.info("Primeras 5 filas obtenidas de PostgreSQL (formato markdown):")

        if not df_from_db.empty:
            print(df_from_db.to_markdown(index=False))
        else:
            logging.info("La consulta no devolvió filas o la tabla está vacía.")

    except Exception as e:
        logging.error(f"Error al consultar datos desde la tabla '{TABLE_NAME}' en PostgreSQL: {e}")
else:
    logging.warning("Motor de base de datos no inicializado. No se puede verificar la tabla.")

2025-05-16 22:04:24,680 - INFO - Consultando las primeras 5 filas de la tabla 'raw_airbnb' desde PostgreSQL.
2025-05-16 22:04:24,684 - INFO - Primeras 5 filas obtenidas de PostgreSQL (formato markdown):


|      id | NAME                                             |     host id | host_identity_verified   | host name   | neighbourhood group   | neighbourhood   |     lat |     long | country       | country code   | instant_bookable   | cancellation_policy   | room type       |   Construction year | price   | service fee   |   minimum nights |   number of reviews | last review   |   reviews per month |   review rate number |   calculated host listings count |   availability 365 | house_rules                                                                                                                                                                                                                                                                                                                                                                                                         | license   |
|--------:|:-------------------------------------------------|------------:|:----------------------

In [11]:
# Celda 11: Finalización (Opcional: cerrar conexiones si es necesario, aunque SQLAlchemy suele manejarlas)
logging.info("Script de carga de datos finalizado.")
if 'engine' in locals() and engine is not None:
    engine.dispose()
    logging.info("Conexiones del motor de SQLAlchemy dispuestas (cerradas).")
else:
    logging.info("Celda 11: Script finalizado.")

2025-05-16 22:04:24,692 - INFO - Script de carga de datos finalizado.
2025-05-16 22:04:24,693 - INFO - Conexiones del motor de SQLAlchemy dispuestas (cerradas).
